In [55]:
#   Table of teachers
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import psycopg2
import datetime
from psycopg2 import Error

import psycopg2
import datetime
from psycopg2 import Error

url = 'https://kgeu.ru/Sveden/Employees'    # link to the table
response = requests.get(url)

soup = bs(response.text, "lxml")
table = soup.find_all('table', class_="table-small table-pps")[0]   # separate table
new_table = pd.DataFrame(columns=range(0,12), index = range(0,482)) # mapping


row_marker = 0  # get data from table
for row in table.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        new_table.iat[row_marker,column_marker] = column.get_text()
        column_marker += 1
    if len(columns) > 0:
        row_marker += 1
delete = [2,3,4,5,6,7,9,10,11]  # deleting unnecessary data

for i in delete:
    new_table = new_table.drop([i], axis= 1)
new_table.rename(columns={0:'id', 1:'name', 8:'subject'}, inplace= True)

if False:   #   True for insert in DB
    try:
        connection = psycopg2.connect(user="postgres",
                                    password="0101",
                                    host="127.0.0.1",
                                    port="5432",
                                    database="Parser")
        for i in range(len(new_table)):
            id = int(new_table['id'][i])
            name = str(new_table['name'][i])
            subject = str(new_table['subject'][i])
            
            cursor = connection.cursor()
            cursor.execute(f"""INSERT INTO public."Prep"(id, name, subj) VALUES ('{id}'::integer, '{name}'::text, '{subject}'::text);""")
            connection.commit()



    except (Exception, Error) as error:
        print("Ошибка при работе с PostgreSQL", error)
    finally:
        if connection:
            cursor.close()
            connection.close()
            print("Соединение с PostgreSQL закрыто")




if False:   # data to .txt
    with open('E:\\Проектики\\Uchebka\\list.txt', 'a') as f:    # route to file ' '
        dfAsString = new_table.to_string(header=False, index=False)
        f.write(dfAsString)



Соединение с PostgreSQL закрыто


In [1]:
# Parse from e.library using table of teachers
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import psycopg2
import datetime
from psycopg2 import Error

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pyautogui as pya
import random
import time
from datetime import date

def authorisation():
    log = driver.find_element(by=By.XPATH, value="""//*[@id="login"]""")    # auto login
    pas = driver.find_element(by=By.XPATH, value="""//*[@id="password"]""")
    log.click()
    log.send_keys("[YOUR LOGIN]")
    sleep = random.uniform(0.3, 1)
    time.sleep(sleep)
    pas.click()
    pas.send_keys("[YOUR PASSWORD]")
    driver.find_element(by=By.XPATH, value="""//*[@id="win_login"]/table[1]/tbody/tr[9]/td/div[2]""").click()
    time.sleep(sleep - 0.11)

    select = Select(driver.find_element(by=By.XPATH, value="""//*[@id="show_param"]/table[2]/tbody/tr[2]/td[1]/div/select"""))  # insert city
    select.select_by_value('Казань')
    time.sleep(sleep - 0.09)

    driver.find_element(by=By.XPATH, value="""//*[@id="show_param"]/table[3]/tbody/tr[2]/td[2]/div""").click()  # KSPEU
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "fancybox-frame")))
    select = Select(driver.find_element(by=By.XPATH, value="""//*[@id="town"]"""))  # insert city
    select.select_by_value('Казань')
    driver.find_element(by=By.XPATH, value="""/html/body/center/form/table[2]/tbody/tr[30]/td[2]/a""").click()
    driver.switch_to.default_content()

def antiBlock():    # random movings
    try:
        pya.moveTo(rand1, rand1 + 31 + rand2, duration= randDur/100)
        pya.moveTo(rand1 + 31 - rand2, rand1, duration= randDur/93)
        pya.moveTo(rand1 + rand2, rand1 - rand2, duration= randDur/130)
        pya.moveTo(rand1 - rand2, rand1 + rand2, duration= randDur/ 85)
    except:
        print("|<->|")

try:
    connection = psycopg2.connect(user="postgres",
                                password="0101",
                                host="127.0.0.1",
                                port="5432",
                                database="Parser")
    
    cursor = connection.cursor()
    cursor.execute("""SELECT id, name, subj FROM public."Prep" """)
    rows = cursor.fetchall()
    options = Options()


    listing = []
    for row in rows:
        listing.append(row[1])
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.get("https://www.elibrary.ru/authors.asp")   # link to elibrary

    authorisation()

    # start of random input
    rand1 = random.randint(200, 1200)
    rand2 = random.randint(50, 150)
    randDur = random.randint(25, 100)
    randCycle = random.randint(1, 3)
    antiBlock()
    # end of random input

    id = 388    # start id of teacher
    start = id
    today = date.today()
    for j in range(start, len(listing)):
        nameB = listing[j]
        id += 1
        name_form = driver.find_element(by=By.XPATH, value="""//*[@id="surname"]""")    # Input data
        name_form.click()
        name_form.clear()
        name_form.send_keys(nameB)
        time.sleep(randDur/100) # not neccessary

        butt = driver.find_element(by=By.XPATH, value="""//*[@id="show_param"]/table[6]/tbody/tr[2]/td[6]/div""")
        butt.click()
        
        # start of random input
        rand1 = random.randint(350, 1000)
        rand2 = random.randint(50, 150)
        randDur = random.randint(25, 100)
        randCycle = random.randint(1, 2)
        for i in range (randCycle):
            antiBlock()
        # end of random input
        
        try:
            try:    # button searching
                WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, """//*[@id="restab"]"""))).click()
                publ = driver.find_element(by=By.CSS_SELECTOR, value="[title^='Список публикаций данного автора в РИНЦ']") # get OutData (.get_attribute("href"))
                citir = driver.find_element(by=By.CSS_SELECTOR, value="[title^='Список цитирований работ данного автора из публикаций, входящих в РИНЦ']")
            except:
                b = 0

            try:
                WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, """//*[@id="restab"]"""))).click()
                publ = driver.find_element(by=By.CSS_SELECTOR, value="[title^='Список публикаций данного автора на портале elibrary.ru']") # get OutData (.get_attribute("href"))
                citir = driver.find_element(by=By.CSS_SELECTOR, value="[title^='Список цитирований работ данного автора из всех публикаций на портале elibrary.ru']")
            except:
                b = 0

            #   insertion of data
            print(f"""INSERT INTO public."Article" (id, public, p_href, citir, c_href, date, title) VALUES ('{id}'::integer, '{int(publ.text)}'::integer, '{publ.get_attribute("href")}'::text, '{int(citir.text)}'::integer, '{citir.get_attribute("href")}'::text, '{today}'::date, '{nameB}'::text)""")
            cursor.execute(f"""INSERT INTO public."Article" (id, public, p_href, citir, c_href, date, title) VALUES ('{id}'::integer, '{int(publ.text)}'::integer, '{publ.get_attribute("href")}'::text, '{int(citir.text)}'::integer, '{citir.get_attribute("href")}'::text, '{today}'::date, '{nameB}'::text)""")
            connection.commit()
            print(str(nameB) + " записан")
        except:
            print(str(nameB) + " нет в базах")
        driver.switch_to.default_content()
        try: 
            authorisation() 
        except:
            print("*auth not nesserable*")
        
        

except (Exception, Error) as error:
    print("Ошибка при работе с PostgreSQL", error)
finally:
    if connection:
        cursor.close()
        connection.close()
        print("Соединение с PostgreSQL закрыто")



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
There is no [win32] chromedriver for browser 101.0.4951 in cache
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\Константин\.wdm\drivers\chromedriver\win32\101.0.4951.41]
C:\Users\D9F8~1\AppData\Local\Temp/ipykernel_8620/243126091.py:72: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


Совин Виктор Евгеньевич нет в базах
